### Analyze outliers in detection/prediction results

In [2]:
import pandas as pd
import numpy as np
import os

In [35]:
def read_prediction_dataframe_from_dir(folder: str, is_tracking=False) -> pd.DataFrame:
    file_list = sorted(os.listdir(folder))
    dfs = []
    for f in file_list:
        #print(f)
        columns=["cls", "x", "y", "w", "h", "conf", "id"] if is_tracking else ["cls", "x", "y", "w", "h", "conf"]

        df = pd.read_csv(os.path.join(folder, f), sep=" ", header=None, names=columns)
        df["frame"] = f
        keypoints_df = df[df["cls"].isin([31, 32])]
        #print(keypoints_df)
        dfs.append(keypoints_df)

    return pd.concat(dfs)

In [36]:
#def convert_single_tracking_result_from_txt(df: pd.DataFrame):
print(os.getcwd())
keypoints_df = read_prediction_dataframe_from_dir(folder="../../runs/detect/predict6/labels", is_tracking=False)
print(len(keypoints_df))
keypoints_df.head()

/home/adam/Projects/DSR/ultimate_analytics/ultimate-pipeline/notebooks/keypoints
33517


,cls,x,y,w,h,conf,frame
0,31,0.699563,0.102965,0.010986,0.019179,0.914857,pony_vs_the_killjoys_pool_013_1.txt
9,31,0.342411,0.104897,0.009255,0.016351,0.699574,pony_vs_the_killjoys_pool_013_1.txt
12,31,0.731543,0.135071,0.010216,0.017926,0.637236,pony_vs_the_killjoys_pool_013_1.txt
13,32,0.519936,0.182404,0.009739,0.016709,0.625701,pony_vs_the_killjoys_pool_013_1.txt
16,31,0.310968,0.135658,0.009590,0.016319,0.448016,pony_vs_the_killjoys_pool_013_1.txt


In [42]:
keypoints_df[keypoints_df["frame"]=="pony_vs_the_killjoys_pool_013_1.txt"]
#keypoints_df[keypoints_df["cls"] == 32]
keypoints_df.head()
print(len(keypoints_df))


33517


In [59]:
def get_ratio_of_frames_with_less_than_n_cones(df: pd.DataFrame, n: int=4) -> float:
    df_filtered = df.groupby('frame').filter(lambda row: len(row[row["cls"]==31]) < n)
    return len(df_filtered['frame'].unique())/len(df['frame'].unique())

In [60]:
ratio = get_ratio_of_frames_with_less_than_n_cones(keypoints_df, n=4)
print(f"Percent of frames with less than 4 keypoints: {ratio*100} %")

Percent of frames with less than 4 keypoints: 70.32063896284292 %


In [46]:
condors_keypoints_df = read_prediction_dataframe_from_dir(folder="../../runs/detect/track2/labels", is_tracking=True)
print(len(condors_keypoints_df))
condors_keypoints_df.head()

19090


,cls,x,y,w,h,conf,id,frame
3,32,0.500280,0.276708,0.010053,0.017431,0.955173,4,machine_vs_condors_pool_006_1.txt
8,32,0.501809,0.323580,0.010426,0.018345,0.941980,9,machine_vs_condors_pool_006_1.txt
13,31,0.299682,0.233273,0.009490,0.017113,0.904370,14,machine_vs_condors_pool_006_1.txt
14,31,0.698093,0.229010,0.010256,0.017395,0.889027,15,machine_vs_condors_pool_006_1.txt
16,31,0.327827,0.200379,0.009752,0.016872,0.867725,17,machine_vs_condors_pool_006_1.txt
